In [0]:
import numpy as np

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add

In [0]:
# Function to create to a series of CONV layers followed by Max pooling layer
def Convblock(channel_dimension, block_no, no_of_convs) :
    Layers = []
    for i in range(no_of_convs) :
        
        Conv_name = "conv"+str(block_no)+"_"+str(i+1)
        
        # A constant kernel size of 3*3 is used for all convolutions
        Layers.append(Convolution2D(channel_dimension,kernel_size = (3,3),padding = "same",activation = "relu",name = Conv_name))
    
    Max_pooling_name = "pool"+str(block_no)
    
    #Addding max pooling layer
    Layers.append(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),name = Max_pooling_name))
    
    return Layers

In [0]:
def FCN_8_helper(image_size):
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = (image_size,image_size,3)))
    
    for l in Convblock(64,1,2) :
        model.add(l)
    
    for l in Convblock(128,2,2):
        model.add(l)
    
    for l in Convblock(256,3,3):
        model.add(l)
    
    for l in Convblock(512,4,3):
        model.add(l)
    
    for l in Convblock(512,5,3):
        model.add(l)
        
    model.add(Convolution2D(4096,kernel_size=(7,7),padding = "same",activation = "relu",name = "fc6"))
      
    #Replacing fully connnected layers of VGG Net using convolutions
    model.add(Convolution2D(4096,kernel_size=(1,1),padding = "same",activation = "relu",name = "fc7"))
    
    # Gives the classifications scores for each of the 21 classes including background
    model.add(Convolution2D(21,kernel_size=(1,1),padding="same",activation="relu",name = "score_fr"))
    
    Conv_size = model.layers[-1].output_shape[2] #16 if image size if 512
    #print(Conv_size)
    
    model.add(Deconvolution2D(21,kernel_size=(4,4),strides = (2,2),padding = "valid",activation=None,name = "score2"))
    
    # O = ((I-K+2*P)/Stride)+1 
    # O = Output dimesnion after convolution
    # I = Input dimnesion
    # K = kernel Size
    # P = Padding
    
    # I = (O-1)*Stride + K 
    Deconv_size = model.layers[-1].output_shape[2] #34 if image size is 512*512
    
    #print(Deconv_size)
    # 2 if image size is 512*512
    Extra = (Deconv_size - 2*Conv_size)
    
    #print(Extra)
    
    #Cropping to get correct size
    model.add(Cropping2D(cropping=((0,Extra),(0,Extra))))
    
    return model
    
    

In [55]:
output = FCN_8_helper(512)
print(len(output.layers))

24


In [56]:
output.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_5 (Permute)          (None, 512, 512, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 512, 512, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 512, 512, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 256, 256, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 256, 256, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 256, 256, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 128, 128, 128)    

In [0]:
def FCN_8(image_size):
    fcn_8 = FCN_8_helper(image_size)
    #Calculating conv size after the sequential block
    #32 if image size is 512*512
    Conv_size = fcn_8.layers[-1].output_shape[2] 
    
    #Conv to be applied on Pool4
    skip_con1 = Convolution2D(21,kernel_size=(1,1),padding = "same",activation=None, name = "score_pool4")
    
    #Addig skip connection which takes adds the output of Max pooling layer 4 to current layer
    Summed = add(inputs = [skip_con1(fcn_8.layers[14].output),fcn_8.layers[-1].output])
    
    #Upsampling output of first skip connection
    x = Deconvolution2D(21,kernel_size=(4,4),strides = (2,2),padding = "valid",activation=None,name = "score4")(Summed)
    x = Cropping2D(cropping=((0,2),(0,2)))(x)
    
    
    #Conv to be applied to pool3
    skip_con2 = Convolution2D(21,kernel_size=(1,1),padding = "same",activation=None, name = "score_pool3")
    
    #Adding skip connection which takes output og Max pooling layer 3 to current layer
    Summed = add(inputs = [skip_con2(fcn_8.layers[10].output),x])
    
    #Final Up convolution which restores the original image size
    Up = Deconvolution2D(21,kernel_size=(16,16),strides = (8,8),
                         padding = "valid",activation = None,name = "upsample")(Summed)
    
    #Cropping the extra part obtained due to transpose convolution
    final = Cropping2D(cropping = ((0,8),(0,8)))(Up)
    softmax = keras.layers.Softmax(axis=-1)(final)
    
    return Model(fcn_8.input, softmax)

In [0]:
import keras

In [0]:
model = FCN_8(512)

In [60]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
permute_6_input (InputLayer)    (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
permute_6 (Permute)             (None, 512, 512, 3)  0           permute_6_input[0][0]            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 512, 512, 64) 1792        permute_6[0][0]                  
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 512, 512, 64) 36928       conv1_1[0][0]                    
____________________________________________________________________________________________

In [0]:
import os
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras import backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [39]:
#这里按自己的directory设置好了。我是mount在collab上以后做的。
#400张被preprocess好的图。np读进来就好。
data_loading_time_start = time.time()
dir_train_img = '/content/drive/My Drive/2007_training_images_size512.npy'
dir_seg_img = '/content/drive/My Drive/2007_segmented_images_size512.npy'
train_images = np.load(dir_train_img)
seg_images_21 = np.load(dir_seg_img)
print(f'Total time to load data: {time.time() - data_loading_time_start}s')
print(train_images.shape)
print(seg_images_21.shape)

Total time to load data: 30.739139795303345s
(632, 512, 512, 3)
(632, 512, 512, 21)


In [0]:
opt = keras.optimizers.adam(lr=0.0003)

In [0]:
model.compile(
    loss='categorical_crossentropy',
    optimizer = opt,
    metrics=['accuracy'])

In [0]:
X = np.array(train_images)
Y = np.array(seg_images_21)

In [0]:
model.fit(X, Y, epochs = 30, batch_size = 8, validation_split=.1)

Train on 568 samples, validate on 64 samples
Epoch 1/30
568/568 [==============================] - 211s 371ms/step - loss: 0.7561 - acc: 0.0632 - val_loss: 0.7859 - val_acc: 0.0432
Epoch 2/30
568/568 [==============================] - 202s 355ms/step - loss: 0.7285 - acc: 0.0668 - val_loss: 0.7584 - val_acc: 0.0449
Epoch 3/30
568/568 [==============================] - 201s 354ms/step - loss: 0.7298 - acc: 0.0675 - val_loss: 0.7618 - val_acc: 0.0449
Epoch 4/30
568/568 [==============================] - 201s 354ms/step - loss: 0.7121 - acc: 0.0675 - val_loss: 0.9041 - val_acc: 0.0447
Epoch 5/30
568/568 [==============================] - 201s 354ms/step - loss: 0.7134 - acc: 0.0673 - val_loss: 0.7600 - val_acc: 0.0449
Epoch 6/30
160/568 [=======>......................] - ETA: 2:18 - loss: 0.6904 - acc: 0.0822

In [0]:
print(model.predict(X[:2]))

[[[[1.64367221e-02 5.24476320e-02 3.47995535e-02 ... 5.42855710e-02
    5.34863733e-02 5.57422750e-02]
   [1.49044693e-02 5.52833900e-02 3.93760540e-02 ... 4.62058671e-02
    6.03263564e-02 4.94036898e-02]
   [1.77825671e-02 5.74673228e-02 3.74481119e-02 ... 5.27243763e-02
    5.09174466e-02 5.85222393e-02]
   ...
   [8.24374706e-03 5.50975986e-02 2.96859052e-02 ... 5.04679084e-02
    5.78288026e-02 6.12867400e-02]
   [1.04155047e-02 6.20361604e-02 2.63831411e-02 ... 4.36967313e-02
    6.13735579e-02 6.87764660e-02]
   [5.25288889e-03 6.27101436e-02 2.47793738e-02 ... 5.11284433e-02
    5.84215447e-02 6.48574904e-02]]

  [[1.77560393e-02 5.60686998e-02 4.25950587e-02 ... 5.48181869e-02
    5.62857725e-02 5.84928244e-02]
   [1.82271600e-02 5.39255925e-02 3.57482508e-02 ... 5.40553406e-02
    4.84387092e-02 5.87901548e-02]
   [1.54601866e-02 5.44060916e-02 3.76634747e-02 ... 5.26953936e-02
    5.66831715e-02 6.05913624e-02]
   ...
   [8.71083327e-03 5.85083105e-02 2.56278478e-02 ... 5.00

In [0]:
model.save('/content/drive/My Drive/FCN_vs.h5')